# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy.engine import create_engine

import nltk
nltk.download(['punkt','wordnet'])
import re
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///disasterpipeline.db')
df = pd.read_sql_table('df', engine)
X = df['message'].values
y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    tokens = word_tokenize(t)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens: 
        clean_tok = lemmatizer.lemmatize(tok.lower())
        clean_tokens.append(clean_tok)
    return clean_tokens
    pass

In [8]:
# test function 
for t in X[:5]:
    tokens = tokenize(t)
    print(t)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'need', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred= pipeline.predict(X_test)

In [55]:
# compile a list of f1_score for all target variables
f1_list = []
for i in range(0,y_test.shape[1]):
   print(classification_report(y_test[:,i],y_pred[:,i]))
   f1_score = float(classification_report(y_test[:,i],y_pred[:,i]).split(" ")[-7])
   f1_list.append(f1_score)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1830
          1       0.77      1.00      0.87      5975

avg / total       0.59      0.77      0.66      7805

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      6466
          1       0.00      0.00      0.00      1339

avg / total       0.69      0.83      0.75      7805

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7777
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      7805

             precision    recall  f1-score   support

          0       0.58      1.00      0.74      4543
          1       0.00      0.00      0.00      3262

avg / total       0.34      0.58      0.43      7805

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      7187
          1       0.00      0.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [61]:
# average out the f1_score as the performance metric for model 
sum(f1_list)/len(f1_list)

0.8916666666666667

### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [63]:
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2,4,6]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose=3)
cv.fit(X_train,y_train)
cv.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.1957166392092257, total=   6.7s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.19818780889621088, total=   6.7s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   20.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.2014827018121911, total=   6.6s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.1957166392092257, total=   8.0s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.19818780889621088, total=   7.9s
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), score=0.2014827018121911, total=   8.1s
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.1957166392092257, total=   6.6s
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.19818780889621088, tot

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  3.3min finished


{'clf__estimator__min_samples_split': 2, 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [64]:
y_pred_cv = cv.predict(X_test)
f1_list_cv = []
for i in range(0,y_test.shape[1]):
   print(classification_report(y_test[:,i],y_pred_cv[:,i]))
   f1_score = float(classification_report(y_test[:,i],y_pred_cv[:,i]).split(" ")[-7])
   f1_list_cv.append(f1_score)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1830
          1       0.77      1.00      0.87      5975

avg / total       0.59      0.77      0.66      7805

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      6466
          1       0.00      0.00      0.00      1339

avg / total       0.69      0.83      0.75      7805

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7777
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      7805

             precision    recall  f1-score   support

          0       0.58      1.00      0.74      4543
          1       0.00      0.00      0.00      3262

avg / total       0.34      0.58      0.43      7805

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      7187
          1       0.00      0.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
# average out the f1_score as the performance metric for model 
sum(f1_list_cv)/len(f1_list_cv)

0.8916666666666667

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [66]:
# run Naive Bayes
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB())),
])

pipeline.fit(X_train,y_train)
y_pred_NB= pipeline.predict(X_test)
f1_list_nb = []
for i in range(0,y_test.shape[1]):
   print(classification_report(y_test[:,i],y_pred_NB[:,i]))
   f1_score = float(classification_report(y_test[:,i],y_pred_NB[:,i]).split(" ")[-7])
   f1_list_nb.append(f1_score)

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


             precision    recall  f1-score   support

          0       0.50      0.00      0.00      1830
          1       0.77      1.00      0.87      5975

avg / total       0.70      0.77      0.66      7805

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      6466
          1       0.21      0.00      0.01      1339

avg / total       0.72      0.83      0.75      7805

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7777
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      7805

             precision    recall  f1-score   support

          0       0.58      0.94      0.72      4543
          1       0.44      0.07      0.12      3262

avg / total       0.52      0.57      0.47      7805

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      7187
          1       1.00      0.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:
sum(f1_list_nb)/len(f1_list_nb)

0.8941666666666666

### 9. Export your model as a pickle file

In [13]:
import pickle
pickle.dump()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.